In [3]:
import soco
print(soco.__version__)

0.23.3


In [4]:
from soco import SoCo, snapshot
import soco
for zone in soco.discover():
    print(zone.player_name)

TypeError: 'NoneType' object is not iterable

In [6]:
soco.discovery.any_soco()

In [9]:
import time

In [10]:
devices = {device.player_name: device for device in soco.discover()}
sp = devices.get('Living Room')

In [66]:
uri = sp.avTransport.GetMediaInfo([("InstanceID", 0)]).get('CurrentURI')

In [67]:
sp.avTransport.GetMediaInfo([("InstanceID", 0)])

{'NrTracks': '13',
 'MediaDuration': 'NOT_IMPLEMENTED',
 'CurrentURI': 'x-sonosapi-radio:sonos%3a3008?sid=303&flags=8296&sn=6',
 'CurrentURIMetaData': '<DIDL-Lite xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:upnp="urn:schemas-upnp-org:metadata-1-0/upnp/" xmlns:r="urn:schemas-rinconnetworks-com:metadata-1-0/" xmlns="urn:schemas-upnp-org:metadata-1-0/DIDL-Lite/"><item id="100c2068sonos%3a3008" parentID="00081004%2fstations%2fen-US%2fNL%2fc2Q6Tkw6ZGFuY2UtcGFydHk" restricted="true"><dc:title>Rap Game</dc:title><upnp:class>object.item.audioItem.audioBroadcast.#list-genre</upnp:class><dc:creator>Current Hip Hop</dc:creator><upnp:albumArtURI>https://sonosradio.imgix.net/placeholders/7c76b528377249ed372c40295700759a_09.png?w=200&amp;auto=format,compress</upnp:albumArtURI><r:albumArtist>Current Hip Hop</r:albumArtist><r:description>Dance Party</r:description><desc id="cdudn" nameSpace="urn:schemas-rinconnetworks-com:metadata-1-0/">SA_RINCON77575_X_#Svc77575-21d7d928-Token</desc></item></DI

In [64]:
uri

'x-sonosapi-stream:tunein%3a4574?sid=303&flags=8224&sn=6'

In [41]:
sp.play_uri(uri)

In [30]:
sp.get_current_transport_info()

{'current_transport_status': 'OK',
 'current_transport_state': 'PLAYING',
 'current_transport_speed': '1'}

In [12]:
url = 'https://www.clubfmserver.be/radioclubfm.mp3'
devices = {device.player_name: device for device in soco.discover()}
sp = devices.get('Living Room')

In [14]:
speaker = 'Living Room'

In [15]:
def new(speaker, url, playtime):
    
    
    try:
        #find all devices and set the seeked one
        devices = {device.player_name: device for device in soco.discover()}
        sp = devices[speaker]
    #     prev_queue = sp.get_queue()
        snap = snapshot.Snapshot(sp, snapshot_queue=True)
        _ = snap.snapshot()
        play_mode = sp.get_current_transport_info()

        if play_mode.get('current_transport_state') == 'PLAYING':
            # save previous settings
    #         prev_vol = sp.volume
            get_pos = int(sp.get_current_track_info().get('playlist_position')) # base is 1
    #         get_seek = sp.get_current_track_info().get('position')

            # play the song
            print('injecting and playing song...')
    #         sp.volume = 3
    #         sp.add_uri_to_queue(url, position=get_pos+1)
    #         sp.next()
    #         sp.play()
            sp.play_uri(url)

            # set play time
            time.sleep(playtime)

            # get back to the old queue
            print('returning to old queue.')
            snap.restore(fade=True)
    #         sp.clear_queue()
    #         sp.add_multiple_to_queue(prev_queue)
    #         sp.volume = prev_vol
    #         sp.play_from_queue(get_pos-1) # base is 0
    #         sp.seek(get_seek)

        else:
            print('playing song...')
            sp.volume = 3
            sp.play_uri(url)
            time.sleep(playtime)
    #         sp.pause()
    #         sp.clear_queue()
    #         sp.add_multiple_to_queue(prev_queue)
            snap.restore()

    except KeyError:
        print(f'speaker {speaker} not found')



In [16]:
new(speaker, url, playtime=3)

injecting and playing song...
returning to old queue.


AttributeError: 'DidlMusicTrack' object has no attribute 'uri'

In [16]:
soco.discover()

{SoCo("192.168.0.28"), SoCo("192.168.0.32"), SoCo("192.168.0.38")}

In [17]:
SoCo("192.168.0.28").play()

In [70]:
# set designated speaker
speaker = 'Living Room'


def play_song(speaker, url, playtime):
    """
    Takes in the string of a speaker name and a playable .mp3 URL file or radio station.
    Injects the song into the playlist and plays it on the designated speaker.
    """

    try:
        #find all devices and set the seeked one
        devices = {device.player_name: device for device in soco.discover()}
        sp = devices[speaker]
        uri = sp.avTransport.GetMediaInfo([("InstanceID", 0)]).get('CurrentURI')
        meta = sp.avTransport.GetMediaInfo([("InstanceID", 0)]).get('CurrentURIMetaData')
        play_mode = sp.get_current_transport_info()
        
        if play_mode.get('current_transport_state') == 'PLAYING':
            if any(x in uri for x in ['radio', 'stream']):
                print('handling radio interruption.')
                # handle radio stations differently, they don't have .next()
                sp.colume = 25
                sp.play_uri(url)
                time.sleep(playtime)
                sp.stop()
                sp.clear_queue()
                sp.play_uri(uri, meta=meta)
                print('restored old radio state.')
        
            else:
                print('handling queue interruption.')
                prev_queue = sp.get_queue()
                play_mode = sp.get_current_transport_info()
                # save previous settings
                prev_vol = sp.volume
                get_pos = int(sp.get_current_track_info().get('playlist_position')) # base is 1
                get_seek = sp.get_current_track_info().get('position')

                # play the song
                print('injecting and playing song...')
                sp.volume = 25
                sp.add_uri_to_queue(url, position=get_pos+1)
                sp.next()
                sp.play()

                # set play time
                time.sleep(playtime)

                # get back to the old queue
                print('returning to old queue.')
                sp.clear_queue()
                sp.add_multiple_to_queue(prev_queue)
                sp.volume = prev_vol
                sp.play_from_queue(get_pos-1) # base is 0
                sp.seek(get_seek)
                print('restored old queue state.')
        
        else:
            print('playing song...')
            sp.volume = 25
            sp.play_uri(url)
            time.sleep(playtime)
            sp.pause()
            sp.clear_queue()
            if 'radio' in uri:
                print('restoring radio to player without playing.')
                sp.play_uri(uri)
                sp.stop()
            else:
                print('restoring old queue to player.')
                sp.add_multiple_to_queue(prev_queue)

    except KeyError:
        print(f'speaker {speaker} not found')
    
play_song(speaker, url, playtime=5)

handling radio interruption.
restored old radio state.


In [23]:
int(of.get_current_track_info().get('playlist_position'))

NameError: name 'of' is not defined

In [91]:
of.add_multiple_to_queue(prev_queue)

In [100]:
prev_queue

Queue(items=[<DidlItem 'b''' at 0x1117d6fa0>, <DidlMusicTrack 'b'klingelstreich.mp3'' at 0x1117d6520>, <DidlItem 'b''' at 0x1117d6070>, <DidlItem 'b''' at 0x1117d68b0>, <DidlItem 'b''' at 0x1117d6490>, <DidlItem 'b''' at 0x1117d6910>, <DidlItem 'b''' at 0x1117d67f0>, <DidlItem 'b''' at 0x1117d6040>, <DidlItem 'b''' at 0x1117d6d30>, <DidlItem 'b''' at 0x1117d6ee0>, <DidlItem 'b''' at 0x11207b910>, <DidlItem 'b''' at 0x11207bbe0>])